In [2]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity


class CollabRecommendation:
    def __init__(self, user_data, product_data, ratings_data):
        self.user_data = user_data
        self.product_data = product_data
        self.ratings_data = ratings_data

    def collaborative_filtering_recommendations(self, target_user_id, top_n=10):
        target_user_id = int(target_user_id)

        user_item_matrix = self.ratings_data.pivot_table(
            index='userid', columns='productid', values='rating', aggfunc='mean'
        ).fillna(0)

        # Ensure target user exists
        if target_user_id not in user_item_matrix.index:
            raise ValueError(f"User ID {target_user_id} not found in the ratings data.")

        target_user_ratings = user_item_matrix.loc[target_user_id]
        rated_items = target_user_ratings[target_user_ratings > 0].index

        similar_users = user_item_matrix.loc[
            (user_item_matrix > 0).any(axis=1) & (user_item_matrix.index != target_user_id)
            ]
        similar_users_similarity = cosine_similarity(
            target_user_ratings.values.reshape(1, -1), similar_users.values
        )[0]

        similar_users_indices = similar_users_similarity.argsort()[::-1]
        recommended_items = []

        for user_index in similar_users_indices[:top_n]:
            similar_user_ratings = similar_users.iloc[user_index]
            recommended_items.extend(similar_user_ratings[similar_user_ratings > 0].index)

        recommended_items = [item for item in set(recommended_items) if item not in rated_items]
        recommended_items_details = self.product_data[[
            'id', 'name']][self.product_data['id'].isin(recommended_items)]

        return recommended_items_details

    def recommend(self, user_id, top_n=20):
        return self.collaborative_filtering_recommendations(user_id, top_n=top_n)

    def save(self, filepath):
        """Save the model to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        """Load the model from a file."""
        with open(filepath, 'rb') as f:
            return pickle.load(f)


# Example usage
if __name__ == "__main__":
    # Load data
    user_data = pd.read_csv('user_data.csv')
    product_data = pd.read_csv('test_data.csv')
    ratings_data = pd.read_csv('ratings_test.csv')

    # Initialize the model
    model = CollabRecommendation(user_data, product_data, ratings_data)

    # Example user ID
    user_id = 64281  # Ensure this matches the correct type (int or str, depending on your data)

    # Generate recommendations
    recommendations = model.recommend(user_id, top_n=10)

    # Display results
    print(f"Recommendations for user ID {user_id}:")
    for _, rec in recommendations.iterrows():
        print(f"Product ID: {rec['id']}, Product Name: {rec['name']}")


Recommendations for user ID 64281:
Product ID: wcl121987, Product Name: Satrani Women's Cotton Silk Saree (807ST4_Pink 1)
Product ID: acs207044, Product Name: Anekaant Women's Sling Bag
Product ID: str242838, Product Name: Skechers Men's Matera White Sneakers-11 UK/India (45 EU)(12 US) (51858-WHT)
Product ID: wcl007924, Product Name: Arayna Women's Cotton Printed Straight Kurti with Palazzo Pants & Dupatta, Floral Dabu
Product ID: acs224747, Product Name: ART ATTENTION Side Comb, Flower Design, Juda Pin Pearl, Hair Pins Comb, Hair Accessory Side Comb for Women and Girls Comb ...
Product ID: app025490, Product Name: UWEK Gold Pre Filter Cartridge, PP Spun Filter 10" inch, 5 Micron, 120 gm Premium Filter Cartridge, Suitable for RO Water ...
Product ID: spf207360, Product Name: COSMUS Polyester Rain Cover with Pouch for School Bags and Backpacks - Navy Blue
Product ID: wcl090952, Product Name: Mittal Enterprises Winter Thermal Toe Skin Color Wool Heavy Warm Ankle Length Women Socks (Pack 